# Raster Averager (X-TREME EDITION 2.0)

This program will take the average of any number of rasters in a directory and output a new raster with all layers averaged.

In [ ]:
#installs
!pip install rasterio
from matplotlib import pyplot
import numpy as np
import rasterio
import PIL
from PIL import Image
from glob import glob
import os

#Allow access to TetonTopo shared drive. The program will not work if this cell is not run.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**PROVIDE THE DIRECTORY OF THE FILES TO BE AVERAGED IN THE NEXT CELL**

IMPORTANT NOTE: Make sure the rasters end in a .tif file extension. You can simply add this extension to the raster files if they have none to begin with. This allows the program to differenciate between raster images and other files in the folder.



In [ ]:
#Import all rasters in a directory
data_dir = '/content/drive/Shareddrives/GIS/NAIC_RxBurn/project_data/Layers/TifToAverage' # This variable is necesscary for the program to find the rasters. it is the path to the directory (folder) holding the geotiffs

#Creates a path list for all raster files to average
#NOTE: Make sure the rasters end in a .tif file extension. You can simply add this extension to the raster files if they have none to begin with. This allows the program to differenciate between raster images and other files in the folder.
file_list = glob(os.path.join(data_dir, '*.tif'))

In [ ]:
#Run this cell to check that the program has the file path for all rasters to be averaged.
print(file_list)

['/content/drive/Shareddrives/GIS/NAIC_RxBurn/project_data/Layers/TifToAverage/Copy of 20240605_1142_HG_NAIC_RxBurnAOI_Overview_LBP_97thP.tif', '/content/drive/Shareddrives/GIS/NAIC_RxBurn/project_data/Layers/TifToAverage/Copy of 20240605_300_HG_NAIC_RxBurnAOI_Overview_LBP_Wind25.tif', '/content/drive/Shareddrives/GIS/NAIC_RxBurn/project_data/Layers/TifToAverage/Copy of 20240605_300_HG_NAIC_RxBurnAOI_Overview_LBP_Wind20.tif', '/content/drive/Shareddrives/GIS/NAIC_RxBurn/project_data/Layers/TifToAverage/Copy of 20240605_300_HG_NAIC_RxBurnAOI_Overview_LBP_Wind15.tif', '/content/drive/Shareddrives/GIS/NAIC_RxBurn/project_data/Layers/TifToAverage/Copy of 20240605_300_HG_NAIC_RxBurnAOI_Overview_LBP_Wind10.tif']


In [ ]:
#Testing to see how many bands the input files contain
for file in file_list:
  with rasterio.open(file) as src:
    print(src.count)

15


15


15


15


15


**The following function takes the file list from above, as well as a file path for the output file (including the name and file extension) as parameters. It then reads in every raster and averages every band of all rasters. The program maintains the meta data by taking the data from the first file in the list and assigning it to the output file. Once this function is called and run the file will be saved to the specified directory.**

In [ ]:
def average_raster_bands(files, output_file):
    try:
        # Open all input raster files
        src_files_to_mosaic = [rasterio.open(file) for file in files]

        # Read metadata of one raster file (assuming all have the same metadata)
        meta = src_files_to_mosaic[0].meta

        # Get number of bands
        num_bands = src_files_to_mosaic[0].count

        # Initialize array to store band data
        band_data = []

        # Read and accumulate band data for each raster file
        for band_index in range(1, num_bands + 1):
            band_data_accumulated = None
            for src in src_files_to_mosaic:
                data = src.read(band_index)
                if band_data_accumulated is None:
                    band_data_accumulated = data.astype(np.float64)
                else:
                    band_data_accumulated += data.astype(np.float64)

            # Calculate average for the current band
            band_data_average = band_data_accumulated / len(src_files_to_mosaic)

            # Append averaged band data to list
            band_data.append(band_data_average.astype(meta['dtype']))

        # Close all input raster files
        for src in src_files_to_mosaic:
            src.close()

        # Write the averaged bands to a new raster file
        with rasterio.open(output_file, 'w', **meta) as dst:
            for i, data in enumerate(band_data):
                dst.write(data, i + 1)

        print(f"Averaged raster file saved as {output_file}")

    except Exception as e:
        print(f"Error processing files: {e}")

**The Following cell assigns the two needed parameters for the function and calls the function. Once you run this cell, the files will be averaged and the new file will be created and stored at the location specified in the `output_file` variable below.

In [ ]:
#File list as created in the above cell.
input_files = file_list

# Output file path and name for the averaged raster (Example: /content/drive/output_folder/filename.tif)
output_file = '/content/drive/Shareddrives/GIS/NAIC_RxBurn/project_data/Layers/AveragedRaster/NAIC_RxBurn_MeanRaster'

average_raster_bands(input_files, output_file)

Averaged raster file saved as /content/drive/Shareddrives/GIS/NAIC_RxBurn/project_data/Layers/AveragedRaster/NAIC_RxBurn_MeanRaster
